In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as web
import datetime as dt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential
import yfinance as yf
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

In [ ]:
# Initialize Firebase
cred = credentials.Certificate('/content/drive/MyDrive/Predictions/financiaflutter-firebase-adminsdk-ajaw7-f515e50a59.json')
firebase_admin.initialize_app(cred)

ValueError: ignored

In [ ]:
crypto_currency = 'RIVN'
against_currency = 'USD'

start = dt.datetime(2016,1,1)
end = dt.datetime.now()

data2 = yf.download('RIVN', start=start, end=end)

[*********************100%***********************]  1 of 1 completed


In [ ]:


scalar= MinMaxScaler(feature_range=(0,1))
scaled_data2 =scalar.fit_transform(data2['Close'].values.reshape(-1,1))

x_train2, y_train2=[], []


for x in range(prediction_days, len(scaled_data2)-future_day):
  x_train2.append(scaled_data2[x-prediction_days:x, 0])
  y_train2.append(scaled_data2[x+future_day, 0])

x_train2, y_train2 = np.array(x_train2), np.array(y_train2)
x_train2 = np.reshape(x_train2, (x_train2.shape[0], x_train2.shape[1], 1))


model2 = Sequential()

model2.add(LSTM(units=50, return_sequences=True, input_shape=(x_train2.shape[1], 1)))
model2.add(Dropout(0.2))
model2.add(LSTM(units=50, return_sequences=True))
model2.add(Dropout(0.2))
model2.add(LSTM(units=50))
model2.add(Dropout(0.2))
model2.add(Dense(units=1))

model2.compile(optimizer='adam', loss='mean_squared_error')
model2.fit(x_train2, y_train2, epochs=25, batch_size=32)

test_data2 = yf.download('RIVN', start=test_start, end=test_end)
actual_prices2 = test_data2['Close'].values

total_dataset2 = pd.concat((data2['Close'], test_data2['Close']), axis=0)


model_inputs2 = total_dataset2[len(total_dataset2) - len(test_data2) - prediction_days:].values
model_inputs2 = model_inputs2.reshape(-1, 1)
model_inputs2 = scalar.fit_transform(model_inputs2)

x_test2 = []

for x in range(prediction_days, len(model_inputs2)):
  x_test2.append(model_inputs2[x-prediction_days:x, 0])

x_test2 = np.array(x_test2)
x_test2 = np.reshape(x_test2, (x_test2.shape[0], x_test2.shape[1], 1))

prediction_prices2 = model2.predict(x_test2)
prediction_prices2 = scalar.inverse_transform(prediction_prices2)

plt.plot(actual_prices2, color='black', label='Actual Prices')
plt.plot(prediction_prices2, color='green', label='Predicted Prices')
plt.title('RIVN price Prediction')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend(loc = 'upper left')
plt.show()

real_data2 = [model_inputs2[len(model_inputs2)+1 - prediction_days:len(model_inputs2) + 1, 0]]
real_data2 = np.array(real_data2)
real_data2 = np.reshape(real_data2, (real_data2.shape[0],real_data2.shape[1], 1))

prediction2 = model2.predict(real_data2)
prediction2 = scalar.inverse_transform(prediction2)
print(prediction2)

NameError: ignored

In [ ]:
# Save the prediction value to Firebase Firestore
db = firestore.client()
doc_ref = db.collection('predictionstock').document(crypto_currency)
doc_ref.set({
    'prediction': float(prediction2[0][0])
})